In [1]:
from qiskit.primitives import Estimator

import scipy.linalg as ln

from povm_toolbox.library import *

from Ring import *

#### Setting up IBM Quantum Experience token

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
    token='feb77ce69166caff54852530c1634b44e307ee4fd5af82cbea78c06774b07ed575a010878366428456e3333366c38aed28bc7a6dd61279eca7febcebc353f3b3'
)

#### Setting the number of sites of the ring ($\equiv$ number of qubit used)

In [3]:
# Ring sites
n_spin = 12

# Creation of the system Hamiltonian as SparsePauliOp
H = build_hamiltonian(n_spin)

#### Generating the annealing circuit

In [5]:
# Number of trotter step -- each trotter step is 15-CNOT deep
n_tstep = 7

# Creation of the annealing circuit
qc = trotterized_annealing(n_spin, n_tstep)

#### Retriving the least busy backend

In [ ]:
backend = service.least_busy(operational=True, min_num_qubits=12)
print(
    f"Name: {backend.name}\n"
    f"Version: {backend.version}\n"
    f"No. of qubits: {backend.num_qubits}\n"
)

In [6]:
backend = service.backend("ibm_brisbane")

In [ ]:
print(type(backend))
print(backend)

In [7]:
# Fixing the qubits to be used on the QPU 
layout = [37, 38, 39, 40 ,41, 53, 60, 59, 58, 57, 56, 52]

# Fanne uno per ognuno degli hardware in modo da utilizzare
# sempre il migliore possibile [TODO]

# Puoi anche investigare il plug_in "mapomatic" che vuole
# automatizzare questo step. [TODO]
# https://github.com/qiskit-community/mapomatic

#### Transpiling the circuit to match the backend

In [8]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
pm = generate_preset_pass_manager(optimization_level=2, backend=backend, initial_layout=layout)

In [9]:
# Transpile the circuit to an "Instruction Set Architecture" (ISA) circuit.
# Note: the transpiler automatically adds "ancilla" qubits to make the transpiled
# circuit match the size of the FakeSherbrooke backend.
isa_circ = pm.run(qc)

# circ.draw()

#### Building the POVM scheme to be used

In [10]:
# Allocating the POVMscheme to use for the expectation value estimation
POVM_scheme = ClassicalShadows(num_qubits=n_spin, measurement_twirl=True, shot_repetitions=5)
# POVM_scheme.measurement_circuit.draw("mpl")

In [ ]:
from POVM import lancia_POVM
# ATTENZIONE:: Da eseguire SOLO per lanciare un nuovo job

# Executing on the choosen backend the POVM
id_file = lancia_POVM(backend, isa_circ, POVM_scheme)

In [11]:
from POVM import recupera_POVM
#ATTENZION:: Da eseguire SOLO per recuperare i risultati di un job terminato

id_file = '20000_2024-10-28 13:47:26.509748'

# Retriving the job results
POVM_results = recupera_POVM(service, id_file)

In [ ]:
POVM_results.metadata.composed_circuit.draw()

#### Execution of the energy estimation throught a Krylov-like Subspace Expansion.
##### Call $|\psi\rangle$ the state obtained by the execution of the annealing circuit, then we build the space $\{\mathrm{I}|\psi\rangle, H|\psi\rangle, H^2|\psi\rangle\}$ and solve the generilized eigenvalue problem $$ \mathcal{H}\nu = \varepsilon\mathcal{S}\nu $$ where $\mathcal{H}_{i,j} = \langle\psi|H^{i+j+1}|\psi\rangle$ and $\mathcal{S}_{i,j} = \langle\psi|H^{i+j}|\psi\rangle$

In [12]:
# Setting the maximum power of the Hamiltonian to use
max_pow = 1

# Mappare le misure delle osservabili al layout del cricuito attuale [TODO]
# Retriving an energy estimation throught digitalized annealing + subspace expansion
estimate = energy_estimate_SbE(isa_circ, H, POVM_results, max_pow)

#### Computing the exact ground state of the Hamiltonian

In [13]:
# Computing a numerical value of the system Hamiltonian
H_num = evaluate_hamiltonian(H)

# Numerical diagonalization of the Hamiltonian
e, V = ln.eigh(np.real(H_num), subset_by_index=[0, 0])

# Recuperiamo l'indice del groung state
id_gs = np.argmin(e)

# Recuperiamo il ground state e la relativa energia
gs     = V[:,id_gs]
energy = e[id_gs]

#### Comparison of the estimate from the Subspace Expansion with the exact (numerical) value of the ground state energy

In [14]:
err = abs(energy - estimate) / abs(energy)
print([energy, estimate, err])

[-5.387390917445205, array([-2.42546954]), array([0.54978772])]
